<a href="https://colab.research.google.com/github/yleessam/recommend/blob/main/movie_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [7]:
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
data = pd.read_csv('tmdb_5000_movies.csv')
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [20]:
data = pd.read_csv('/content/gdrive/MyDrive/!assist/7.rec/tmdb_5000_movies.csv')

In [47]:
data[data['title']=='The Dark Knight Rises']

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
3,49026,Action Crime Drama Thriller,7.6,9106,112.31295,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...


In [21]:
data.shape

(4803, 20)

In [22]:
list(data)

['budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'vote_average',
 'vote_count']

In [23]:
data = data[['id', 'genres', 'vote_average','vote_count', 'popularity', 'title', 'keywords', 'overview']]
data

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...
4798,9367,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",6.6,238,14.269792,El Mariachi,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...
4799,72766,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5.9,5,0.642552,Newlyweds,[],A newlywed couple's honeymoon is upended by th...
4800,231617,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",7.0,6,1.444476,"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,126186,[],5.7,7,0.857008,Shanghai Calling,[],When ambitious New York attorney Sam is sent t...


In [24]:
#투표 상위 10%를 위한 최소 투표수 확인
tmp_m = data['vote_count'].quantile(0.89)
tmp_m

1683.8999999999987

In [25]:
m = data['vote_count'].quantile(0.9)
data = data.loc[data['vote_count']>=m]
data.shape

(481, 8)

In [26]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."


In [27]:
data['genres']= data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

<ipython-input-27-aed9f1533311>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres']= data['genres'].apply(literal_eval)
<ipython-input-27-aed9f1533311>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywords'] = data['keywords'].apply(literal_eval)


In [28]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."


In [29]:
data['genres'] = data['genres'].apply(lambda x: [d['name'] for d in x ]).apply(lambda x:" ".join(x))
data['keywords'] = data['keywords'].apply(lambda x: [d['name'] for d in x ]).apply(lambda x:" ".join(x))


<ipython-input-29-ed43c2c3b2ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres'] = data['genres'].apply(lambda x: [d['name'] for d in x ]).apply(lambda x:" ".join(x))
<ipython-input-29-ed43c2c3b2ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywords'] = data['keywords'].apply(lambda x: [d['name'] for d in x ]).apply(lambda x:" ".join(x))


In [30]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di..."
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha..."


In [31]:
data.genres.head(2)

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
Name: genres, dtype: object

In [33]:
count_vector = CountVectorizer(ngram_range=(1,3))
count_vector

CountVectorizer(ngram_range=(1, 3))

In [34]:
c_vector_genres = count_vector.fit_transform(data['genres'])
c_vector_genres

<481x364 sparse matrix of type '<class 'numpy.int64'>'
	with 3238 stored elements in Compressed Sparse Row format>

In [35]:
c_vector_genres.shape

(481, 364)

In [36]:
c_vector_genres.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [37]:
v_vector_genres = count_vector.fit(data['genres'])

In [38]:
v_vector_genres.vocabulary_

{'action': 0,
 'adventure': 48,
 'fantasy': 225,
 'science': 326,
 'fiction': 259,
 'action adventure': 1,
 'adventure fantasy': 79,
 'fantasy science': 256,
 'science fiction': 327,
 'action adventure fantasy': 5,
 'adventure fantasy science': 86,
 'fantasy science fiction': 257,
 'fantasy action': 226,
 'adventure fantasy action': 80,
 'crime': 143,
 'adventure crime': 65,
 'action adventure crime': 4,
 'drama': 159,
 'thriller': 336,
 'action crime': 15,
 'crime drama': 149,
 'drama thriller': 193,
 'action crime drama': 16,
 'crime drama thriller': 152,
 'adventure science': 88,
 'action adventure science': 6,
 'adventure science fiction': 89,
 'fantasy action adventure': 227,
 'animation': 94,
 'family': 201,
 'animation family': 105,
 'fantasy family': 250,
 'adventure fantasy family': 84,
 'adventure action': 49,
 'action thriller': 39,
 'thriller crime': 343,
 'adventure action thriller': 54,
 'action thriller crime': 41,
 'western': 360,
 'adventure western': 93,
 'action adve

In [40]:
#코사인 유사도를 구한 벡터를 저장
genre_c_sim =  cosine_similarity(c_vector_genres, c_vector_genres)

In [41]:
genre_c_sim

array([[1.        , 0.47140452, 0.35355339, ..., 0.        , 0.        ,
        0.        ],
       [0.47140452, 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.35355339, 0.33333333, 1.        , ..., 0.        , 0.23570226,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.23570226, ..., 0.        , 1.        ,
        0.57735027],
       [0.        , 0.        , 0.        , ..., 0.        , 0.57735027,
        1.        ]])

In [42]:
genre_c_sim[0]

array([1.        , 0.47140452, 0.35355339, 0.09622504, 0.57735027,
       0.47140452, 0.        , 0.57735027, 0.35355339, 0.70710678,
       0.19245009, 0.47140452, 0.35355339, 1.        , 0.57735027,
       0.35355339, 0.38490018, 0.70710678, 0.70710678, 0.5       ,
       0.23570226, 0.        , 0.48112522, 0.5       , 0.5       ,
       0.35355339, 0.70710678, 0.57735027, 0.23570226, 0.41666667,
       0.        , 0.57735027, 0.57735027, 0.23570226, 0.70710678,
       0.48112522, 0.09622504, 0.41666667, 0.        , 0.38490018,
       0.28867513, 0.2981424 , 1.        , 0.57735027, 0.23570226,
       0.        , 0.38490018, 0.48112522, 0.48112522, 0.33333333,
       0.2236068 , 0.57735027, 0.        , 0.57735027, 0.58333333,
       0.5       , 0.23570226, 0.5       , 0.09622504, 0.09622504,
       0.48112522, 0.11785113, 0.67082039, 0.47140452, 0.41666667,
       0.        , 0.19245009, 0.48112522, 0.35355339, 0.25      ,
       0.33333333, 0.57735027, 0.25      , 0.33333333, 0.16666

In [43]:
genre_c_sim = genre_c_sim.argsort()[:, ::-1]

In [44]:
genre_c_sim.shape

(481, 481)

In [45]:
genre_c_sim

array([[  0,  13,  42, ..., 298, 297, 240],
       [ 11,   1, 200, ..., 329, 330, 240],
       [  2, 376, 216, ..., 314, 304, 240],
       ...,
       [478, 187,  12, ..., 326, 327,   0],
       [479, 466, 383, ..., 220, 224,   0],
       [480, 468, 294, ..., 246, 248,   0]])

In [46]:
data[data['title']=='The Dark Knight Rises']

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
3,49026,Action Crime Drama Thriller,7.6,9106,112.31295,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...


In [53]:
def get_recommend_movie_list(df, movie_title, top=30):
  target_movie_index= df[df['title']==movie_title].index.values
  sim_index  = genre_c_sim[target_movie_index,:top].reshape(-1)
  sim_index = sim_index[sim_index!= target_movie_index]
  result = df.iloc[sim_index].sort_values('vote_average', ascending=False)[:top]
  return result

In [54]:
get_recommend_movie_list(data, movie_title='The Dark Knight Rises', top=10)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ..."
2760,264644,Drama Thriller,8.1,2757,66.113340,Room,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...
1850,111,Action Crime Drama Thriller,8.0,2948,70.105981,Scarface,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...
1829,6977,Crime Drama Thriller,7.7,3003,53.645267,No Country for Old Men,texas drug traffic hitman united states–mexico...,"Llewelyn Moss stumbles upon dead bodies, $2 mi..."
3234,242582,Crime Drama Thriller,7.6,3395,89.095538,Nightcrawler,journalism underground tv station sociopath ho...,"When Lou Bloom, desperate for work, muscles in..."
119,272,Action Crime Drama,7.5,7359,115.040024,Batman Begins,himalaya martial arts dc comics crime fighter ...,"Driven by tragedy, billionaire Bruce Wayne ded..."
2730,44214,Drama Thriller,7.3,4430,91.285683,Black Swan,dancing competition sex dancer obsession paran...,"A ballet dancer wins the lead in ""Swan Lake"" a..."
1596,273481,Action Crime Drama Mystery Thriller,7.2,2416,55.424027,Sicario,mexico cia smoking texas fbi murder dirty cop ...,A young female FBI agent joins a secret CIA op...
717,75780,Crime Drama Thriller,6.3,2998,56.868399,Jack Reacher,based on novel sniper investigation police quarry,"In an innocent heartland city, five are shot d..."


In [55]:
data[data['title'] == 'The Dark Knight Rises']

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
3,49026,Action Crime Drama Thriller,7.6,9106,112.31295,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...
